# Análisis de patrones de éxito para la industria de los videojuegos

# Contenido

* [Introducción](#)
* [Objetivos](#)
* [Etapas](#)
* [Inicialización](#)
* [Preprocesamiento de datos](#)
* [Análisis exploratorio de datos](#)
* [Pruebas de hipótesis](#)
* [Conclusiones](#)

# Introducción

En esta oportunidad la tienda online dedicada a la venta de videojuegos *Ice* nos tiene por encargo analizar e indentificar patrones que determinen si un juego tiene éxito o no, para luego detectar proyectos prometedores y planificar campañas publicitarias al respecto. Para esto se nos proporciona una base de datos de hasta 2016, incluyendo reseñas de usuarios y expertos, géneros, plataformas, etc. Con el fin de lograr estos objetivos, haremos usos de las diversas técnicas de preprocesamiento, análisis exploratorio y estadístico de los datos, para así abordar de la forma más integral posible el proceso analítico.

Las hipótesis estadísticas específicas serán presentadas en el siguiente apartado.

# Objetivos

Tenemos, en particular, estas hipótesis específicas a probar:

* Existe diferencia entre las calificaciones promedio de los usuarios para las plataformas Xbox One y PC.
* Existe diferencia entre las calificaciones promedio de los usuarios para los géneros de Acción y Deportes.

A estas hipótesis estadísticas complementamos con analítica general sobre las ventas, reseñas de usuarios, comportamiento del usuario, plataformas y regiones.

# Etapas de análisis

Para este proyecto, seguiremos el siguiente esquema básico para mantener el proceso ordenado:
1. Descripción de datos
2. Preprocesamientos de datos
3. Análisis exploratorio de datos
4. Prueba de hipótesis
5. Conclusiones

Debido a la asimetría de información, para garantizar un adecuado tratamiento y análisis de datos se usarán algunos supuestos razonables, cuya lógica será explicada y no debería deteriorar nuestras conclusiones.

# Inicialización: descripción de datos

**Carga de librerías necesarias**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
# carga todas las librerías necesarias

## Cargar los datos

In [4]:
try:
    games = pd.read_csv("games.csv")
except:
    games = pd.read_csv("/datasets/games.csv")
# carga del dataset

## Exploración inicial de datos

En este dataset, la información que se tiene dentro del dataset es la siguiente:

* ``Name``: nombre de videojuego.
* ``Platform``: nombre de la plataforma del juego.
* ``Year_of_Release``: año de lanzamiento (del videojuego).
* ``Genre``: género del videojuego.
* ``NA_sales``: ventas en Norteamérica en millones de dólares estadounidenses.
* ``EU_sales``: ventas en Europa en millones de dólares estadounidenses.
* ``JP_sales``: ventas en Japón en millones de dólares estadounidenses.
* ``Other_sales``: ventas en otros países en millones de dólares estadounidenses.
* ``Critic_Score``: puntaje de la reseña profesional, máximo de 100.
* ``User_Score``: puntaje de la reseña de los usuarios, máximo de 10.
* ``Rating``: clasificación ESRB.

Con esto en mente, obtendremos la información inicial del DataFrame:

In [6]:
games.describe()

,Year_of_Release,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score
count,16446.000000,16715.000000,16715.000000,16715.000000,16715.000000,8137.000000
mean,2006.484616,0.263377,0.145060,0.077617,0.047342,68.967679
std,5.877050,0.813604,0.503339,0.308853,0.186731,13.938165
min,1980.000000,0.000000,0.000000,0.000000,0.000000,13.000000
25%,2003.000000,0.000000,0.000000,0.000000,0.000000,60.000000
50%,2007.000000,0.080000,0.020000,0.000000,0.010000,71.000000
75%,2010.000000,0.240000,0.110000,0.040000,0.030000,79.000000
max,2016.000000,41.360000,28.960000,10.220000,10.570000,98.000000


In [7]:
games.head(10)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


In [5]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Como primera observación, todas las columnas tienen nombres que incluyen mayúsculas, detalle que no es recomendable. Ahora detallaremos los hallazgos por columnas:

* ``Name`` presenta sus valores con mayúsculas y minúsculas, además de tener dos valores ausentes.
* ``Platform`` también presenta valores con mayúsculas y minúsculas.
* ``Year_of_Release`` está registrado como tipo flotante (incorrectamente) y tiene algunos valores ausentes.
* ``Genre`` tiene valores en mayúsculas y dos valores ausentes.
* ``NA_sales``, ``EU_sales``, ``JP_sales`` y ``Other_sales`` no parecen tener problemas explícitos.
* ``Critic_Score`` presenta un gran porcentaje de valores ausentes.
* ``User_Score`` tiene valores ausentes, aunque en menor proporción que la columna anterior y está registrado como tipo objeto, lo cual podría indicar que hay cadenas dentro de esta columna.
* ``Rating`` también presenta una relativa gran cantidad de valores ausentes, así como sus valores en mayúsculas.

Dados estos resultados, daremos un repaso a los valores únicos de las columnas categóricas como paso a continuación:

In [10]:
games_categ = games[['Name', 'Platform', 'Genre', 'Rating']]
for column in games_categ:
    try:
        print(f"Valores únicos de la columna '{column}':") # muestra el nombre
        print(games_categ[column].unique()) # valores únicos de cada columna
        print(f"Total: {len(games_categ[column].unique())}")
        print()
    except:
        print("Error al mostrar valores únicos de games_categ")

Valores únicos de la columna 'Name':
['Wii Sports' 'Super Mario Bros.' 'Mario Kart Wii' ...
 'Woody Woodpecker in Crazy Castle 5' 'LMA Manager 2007'
 'Haitaka no Psychedelica']
Total: 11560

Valores únicos de la columna 'Platform':
['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
Total: 31

Valores únicos de la columna 'Genre':
['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' nan]
Total: 13

Valores únicos de la columna 'Rating':
['E' nan 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']
Total: 9



Tal como ya habíamos notado, todas las columnas presentan valores con uso de mayúsculas, así como algunas con valores ausentes. Fuera de esto, destacamos lo siguiente:
* Tenemos 11560 juegos distintos, en teoría, ya que se tiene en cuenta a los valores ausentes.
* Tenemos 31 plataformas de juego en los registros históricos.
* Tenemos 12 tipos o géneros de juegos; incluyendo los ausentes son 13 valores únicos.
* La clasificación se resume en 8 tipos; teniendo 9 valores únicos al considerar los ausentes.

Debido a nuestro proceso de análisis, debemos describir la clasificación ESRB y entender qué significa cada categoría o nomenclatura dada, la cual es la siguiente:

*  ``RP``: (Rating Pending) indica que el respectivo juego aún no tiene una calificación asignada por ESRB.
* ``E``: (Everyone) indica contenido apto para todas las edades.
* ``K-A``: (Kids to Adults) indica contenido apto para todas las edades, utilizado hasta 1998 y reemplazado por ``E``.
* ``E10+``: (Everyone 10+) indica contenido apto para mayores de 10 años.
* ``T``: (Teen) indica contenido apto para mayores de 13 años.
* ``M``: (Mature) indica contenido apto para edades de 17 a más.
* ``AO``: (Adults Only) indica contenido solo indicado a mayores de 18 años.
* ``EC``: (Early Childhood) indicaba contenido para audiencia preescolar, retirado en 2018.

Para analizar la distribución de estas columnas, haremos llamado del método "value_counts":

In [11]:
for column in games_categ:
    try:
        print(f"Distribución de valores de la columna '{column}':")
        print(games_categ[column].value_counts(dropna=False, normalize=True))
        print()
    except:
        print("Error al mostrar la distribución de columnas categóricas")

Distribución de valores de la columna 'Name':
Need for Speed: Most Wanted                         0.000718
Madden NFL 07                                       0.000538
FIFA 14                                             0.000538
Ratatouille                                         0.000538
LEGO Marvel Super Heroes                            0.000538
                                                      ...   
Jewels of the Tropical Lost Island                  0.000060
Sherlock Holmes and the Mystery of Osborne House    0.000060
The King of Fighters '95 (CD)                       0.000060
Megamind: Mega Team Unite                           0.000060
Haitaka no Psychedelica                             0.000060
Name: Name, Length: 11560, dtype: float64

Distribución de valores de la columna 'Platform':
PS2     0.129285
DS      0.128687
PS3     0.079629
Wii     0.078971
X360    0.075501
PSP     0.072330
PS      0.071612
PC      0.058271
XB      0.049297
GBA     0.049177
GC      0.033264
3DS

De las distribuciones destacamos la gran relevancia para juegos en plataformas de PS2 y Nintendo DS, ambas con más del 12 % de cuota cada una, relativamente alta respecto a las demás plataformas. En ``Genre``, el género más frecuente es el de "action". Para el caso de ``Rating``, los valores ausentes representan más del 40 % de los valores de la columna. Este caso merece especial atención en el preprocesamiento de datos. No parecen existir más problemas por ahora en nuestro DataFrame.

Dado que el análisis de variables numéricas y categóricas está realizado, ahora exploraremos un poco más de los valores ausentes.

## Exploración de los valores ausentes

Lo primero será realizar un conteo de los valores ausentes por columnas en términos porcentuales y determinar el porcentaje de valores ausentes para cada columna:

In [17]:
(games.isna().sum()/len(games))*100

Name                0.011965
Platform            0.000000
Year_of_Release     1.609333
Genre               0.011965
NA_sales            0.000000
EU_sales            0.000000
JP_sales            0.000000
Other_sales         0.000000
Critic_Score       51.319174
User_Score         40.089740
Rating             40.478612
dtype: float64

Las columnas ``Name`` y ``Genre`` tienen el mismo porcentaje de valores ausentes debido a que, como recordamos, tienen dos valores ausentes cada una. ``Year_of_Release`` presenta un 1.6 % de valores ausentes. Los casos más críticos que requieren tratamiento son ``Critic_Score`` (51.3 %), ``User_Score`` (40.01 %) y ``Rating`` (40.48 %), porque parte de nuestro análisis requerirá tomar en cuenta las reseñas y clasificación. 

Para analizar si los valores ausentes de algunas de estas últimas cuatro columnas tienen relación con alguna otra columna de nuestro DataFrame, mostraremos las distribuciones del dataset completo y las distribuciones sin valores ausentes de cada columna, de la manera siguiente:

In [36]:
for column in games:
    print(games[column].value_counts(dropna=False, normalize=True))
    print()

Need for Speed: Most Wanted                         0.000718
Madden NFL 07                                       0.000538
FIFA 14                                             0.000538
Ratatouille                                         0.000538
LEGO Marvel Super Heroes                            0.000538
                                                      ...   
Jewels of the Tropical Lost Island                  0.000060
Sherlock Holmes and the Mystery of Osborne House    0.000060
The King of Fighters '95 (CD)                       0.000060
Megamind: Mega Team Unite                           0.000060
Haitaka no Psychedelica                             0.000060
Name: Name, Length: 11560, dtype: float64

PS2     0.129285
DS      0.128687
PS3     0.079629
Wii     0.078971
X360    0.075501
PSP     0.072330
PS      0.071612
PC      0.058271
XB      0.049297
GBA     0.049177
GC      0.033264
3DS     0.031110
PSV     0.025725
PS4     0.023452
N64     0.019085
XOne    0.014777
SNES    0.0142

Ahora analizaremos (sin) los valores ausentes de ``Year_of_Release``:

In [37]:
for column in games.dropna(subset='Year_of_Release'):
    print(games.dropna(subset='Year_of_Release')[column].value_counts(dropna=False, normalize=True))
    print()

Need for Speed: Most Wanted               0.000730
Ratatouille                               0.000547
FIFA 14                                   0.000547
LEGO Marvel Super Heroes                  0.000547
FIFA 15                                   0.000486
                                            ...   
Let's Make a Soccer Team!                 0.000061
Loving Life with Hello Kitty & Friends    0.000061
Scrabble (Others sales)                   0.000061
Viva Pinata: Party Animals                0.000061
Haitaka no Psychedelica                   0.000061
Name: Name, Length: 11427, dtype: float64

PS2     0.129332
DS      0.128968
PS3     0.079411
Wii     0.078195
X360    0.074912
PSP     0.072540
PS      0.072358
PC      0.058190
GBA     0.049313
XB      0.048826
GC      0.032956
3DS     0.031132
PSV     0.026085
PS4     0.023836
N64     0.019214
XOne    0.015019
SNES    0.014532
SAT     0.010519
WiiU    0.008938
2600    0.007053
NES     0.005959
GB      0.005898
DC      0.003162
GEN  

Después de una revisión, no se notaron cambios significativos en las distribuciones de las columnas al eliminar los ausentes de ``Year_of_Release``. Ahora continuamos con ``Critic_Score`` (sin sus valores ausentes):

In [38]:
for column in games.dropna(subset='Critic_Score'):
    print(games.dropna(subset='Critic_Score')[column].value_counts(dropna=False, normalize=True))
    print()

Madden NFL 07                                0.001106
LEGO Star Wars II: The Original Trilogy      0.000983
Madden NFL 08                                0.000983
Need for Speed: Most Wanted                  0.000983
Cars                                         0.000983
                                               ...   
Dr. Mario / Puzzle League                    0.000123
DECA Sports Freedom                          0.000123
The Witcher: Enhanced Edition                0.000123
Mystery Case Files: The Malgrave Incident    0.000123
15 Days                                      0.000123
Name: Name, Length: 5085, dtype: float64

PS2     0.159518
X360    0.112572
PS3     0.100774
XB      0.089099
DS      0.088116
PC      0.087870
Wii     0.071894
PSP     0.056778
GC      0.055057
GBA     0.053828
PS4     0.030970
PS      0.024579
XOne    0.020769
3DS     0.020646
PSV     0.014747
WiiU    0.011061
DC      0.001721
Name: Platform, dtype: float64

2008.0    0.087870
2007.0    0.085044
2005.

Los cambios más notables están en la columna referida al nombre del videojuego, donde el orden de importancia cambia un poco y en las plataformas, donde también el orden cambia un poco. Pero estos cambios eran de esperarse dada la magnitud de los valores ausentes de esta columna y a la vez no son cambios muy grandes. Es decir, no se puede considerar a estos cambios un patrón de por sí. Ahora continuamos con ``User_Score``.

In [39]:
for column in games.dropna(subset='User_Score'):
    print(games.dropna(subset='User_Score')[column].value_counts(dropna=False, normalize=True))
    print()

Madden NFL 07                   0.000899
Need for Speed: Most Wanted     0.000899
LEGO Marvel Super Heroes        0.000799
FIFA Soccer 13                  0.000799
Ratatouille                     0.000799
                                  ...   
Virtua Fighter 5 Online         0.000100
Deer Hunter                     0.000100
Wappy Dog                       0.000100
Summer Heat Beach Volleyball    0.000100
15 Days                         0.000100
Name: Name, Length: 6119, dtype: float64

PS2     0.147893
DS      0.127022
X360    0.104853
Wii     0.099860
PS3     0.096265
PC      0.081786
XB      0.073497
PSP     0.054324
GBA     0.051628
GC      0.046834
PS4     0.026263
3DS     0.022968
PS      0.020671
XOne    0.019273
PSV     0.014979
WiiU    0.010485
DC      0.001398
Name: Platform, dtype: float64

2009.0    0.101758
2008.0    0.098562
2007.0    0.081985
2010.0    0.079688
2011.0    0.071700
2005.0    0.070302
2006.0    0.067505
2002.0    0.065508
2003.0    0.061015
2004.0    0.059

De igual forma que con la anterior columna, los cambios registrados en algunas columnas no compensan la proporción de valores ausentes eliminados en esta columna. Por eso no se considera un patrón como tal. 

Finalizamos con los valores ausentes de ``Rating``.

In [40]:
for column in games.dropna(subset='Rating'):
    print(games.dropna(subset='Rating')[column].value_counts(dropna=False, normalize=True))
    print()

Need for Speed: Most Wanted         0.000905
Madden NFL 07                       0.000905
Madden NFL 08                       0.000804
FIFA 14                             0.000804
LEGO Jurassic World                 0.000804
                                      ...   
Silent Hill 2: Restless Dreams      0.000101
NES Remix                           0.000101
Bass Strike                         0.000101
Medieval Moves: Deadmund's Quest    0.000101
Plushees                            0.000101
Name: Name, Length: 6066, dtype: float64

PS2     0.148859
DS      0.127953
X360    0.105739
Wii     0.100714
PS3     0.095688
PC      0.077797
XB      0.073676
PSP     0.054679
GBA     0.052468
GC      0.047140
PS4     0.025631
3DS     0.022917
PS      0.020907
XOne    0.018695
PSV     0.015177
WiiU    0.010554
DC      0.001407
Name: Platform, dtype: float64

2009.0    0.101618
2008.0    0.098603
2007.0    0.082521
2010.0    0.080310
2005.0    0.070962
2011.0    0.070660
2006.0    0.068148
2002.0   

El mismo fenómeno ocurre con los valores ausentes en esta columna, que finalmente no termina por mostrar un patrón real. Por tanto, concluimos que no existe una relación entre los valores ausentes de nuestro DataFrame con alguna de sus columnas.

## Conclusiones del apartado

Luego de realizado el primer proceso exploratorio inicial del dataset, podemos resumir los hallazgos en los siguientes puntos:

* Nuestro dataset tiene títulos y muchos valores de sus columnas con uso de mayúsculas y minúsculas.
* Algunas columnas (como ``Year_of_Release``) tienen tipos de datos incorrectos que necesitan correciones.
* Cuatro columnas de nuestro dataset cuentan con problemas severos de valores ausentes: ``Year_of_Release``, ``Critic_Score``, ``User_Score`` y ``Rating``, mientras que en otras dos el problema se puede considerar leve.
* Los valores ausentes de las columnas mencionadas no parecen mostrar un patrón o relación con los valores de las demás columnas. 

Dados estos resultados, a continuación pasaremos a la solución de estos problemas: preprocesamiento de datos.

# Preprocesamiento de datos

El primer paso antes de entrar a los valores de las columnas es solucionar el problema con los nombres de las columnas en sí:

## Corregir nombres de las columnas

Usaremos el formato "string.lower" para solucionar este problema:

In [41]:
games.columns = games.columns.str.lower()
games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

## Corregir datos de ``name``

En esta columna detectamos dos valores ausentes, los cuales esta vez analizaremos a profundidad filtrándolos.

In [42]:
games[games['name'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Notamos que buena parte de sus valores son también nulos. Analizando un poco, no tenemos manera de tratar estos datos por la pobre cantidad de información que se tiene. Dado el poco impacto que tendría, decidimos prescindir de estas filas:

In [43]:
games = games.drop([659, 14244]).reset_index(drop=True)
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16713 entries, 0 to 16712
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB
